### Выгрузка данных

In [29]:
import pandas as pd
import numpy as np
import OpenBlender
import json
token = "your_token"
action = 'API_getObservationsFromDataset'
# ANCHOR: 'Bitcoin vs USD'
  
parameters = { 
    'token' : token,
    'id_dataset' : '5d4c3af79516290b01c83f51',
    'date_filter':{"start_date" : "2020-01-01",
                   "end_date" : "2020-08-29"} 
}
df = pd.read_json(json.dumps(OpenBlender.call(action, parameters)['sample']), convert_dates=False, convert_axes=False).sort_values('timestamp', ascending=False)
df.reset_index(drop=True, inplace=True)
df['date'] = [OpenBlender.unixToDate(ts, timezone = 'GMT') for ts in df.timestamp]
df = df.drop('timestamp', axis = 1)

df['log_diff'] = np.log(df['price']) - np.log(df['open'])
df['target'] = [1 if log_diff > 0 else 0 for log_diff in df['log_diff']]

data_format = '%d-%m-%Y %H:%M:%S'
timezone = 'GMT'
df['u_timestamp'] = OpenBlender.dateToUnix(df['date'], 
                                           date_format = data_format, 
                                           timezone = timezone)
df = df[['date', 'u_timestamp', 'price', 'target']]
df = df.rename(columns={"u_timestamp": "timestamp"})
df.head()

Task ID: '6167ff770895fafb4a9d8d4b'.
Total estimated consumption: 500.02 processing units.
Task confirmed. Starting download..
100.0 % completed.


,date,timestamp,price,target
0,28-08-2020 17:00:00,1.598634e+09,11545.0,1
1,27-08-2020 17:00:00,1.598548e+09,11343.0,0
2,26-08-2020 17:00:00,1.598461e+09,11471.0,1
3,25-08-2020 17:00:00,1.598375e+09,11341.0,0
4,23-08-2020 17:00:00,1.598202e+09,11663.0,0


In [30]:
search_keyword = 'bitcoin'
df = df.sort_values('timestamp').reset_index(drop = True)
print('From : ' + OpenBlender.unixToDate(min(df.timestamp)))
print('Until: ' + OpenBlender.unixToDate(max(df.timestamp)))
search_results = OpenBlender.searchTimeBlends(token, df.timestamp, search_keyword)

From : 01-01-2020 17:00:00
Until: 28-08-2020 17:00:00


In [31]:
# We need to add the 'id_dataset' and the 'feature' name we want.

blend_source = {
                'id_dataset':'5ea2039095162936337156c9',
                'feature' : 'text'
            }

# Now, let's 'timeBlend' it to our dataset
df_blend = OpenBlender.timeBlend( token = token,
                                  anchor_ts = df.timestamp,
                                  blend_source = blend_source,
                                  blend_type = 'agg_in_intervals',
                                  interval_size = 60 * 60 * 24,
                                  direction = 'time_prior',
                                  interval_output = 'list',
                                  missing_values = 'raw')
df = pd.concat([df, df_blend.loc[:, df_blend.columns != 'timestamp']], axis = 1)
df.head()

Task ID: '6167ff7c0895fafb4a9d8d4c'.
Total estimated consumption: 1555.2 processing units.
Task confirmed. Starting download..
100%


,date,timestamp,price,target,BITCOIN_NE.text_COUNT_last1days,BITCOIN_NE.text_last1days
0,01-01-2020 17:00:00,1.577898e+09,7212.7,1,3,[etoro ceo yoni assia on reaching 12 million u...
1,02-01-2020 17:00:00,1.577984e+09,6989.4,0,7,[government confirms crypto profits not taxabl...
2,03-01-2020 17:00:00,1.578071e+09,7367.5,1,8,[ticker tool uses the bch blockchain to provid...
3,04-01-2020 17:00:00,1.578157e+09,7369.8,0,6,[crypto fundraising is changing again in 2020 ...
4,05-01-2020 17:00:00,1.578244e+09,7364.6,0,5,[subhash chandra garg on the future of crypto ...


In [32]:
df_blend['BITCOIN_NE.text_COUNT_last1days'].mean()

5.677966101694915

In [33]:
from io import StringIO

action = 'API_getObservationsFromDataset'

# ANCHOR: 'CoinJournal Tweet'

        
parameters = { 
    	'token': "your_token",
        'id_user':'6151d8659516292e82e51d85',
        'id_dataset':'5ea20b4e95162936348f141d' 
}
        

df_v2 = pd.read_json(StringIO(json.dumps(OpenBlender.call(action, parameters)['sample'])), convert_dates=False, convert_axes=False).sort_values('timestamp', ascending=False)
df_v2.reset_index(drop=True, inplace=True)
df_v2.head()

Task ID: '6167ff800895fafb4a9d8d4b'.
Total estimated consumption: 500.36 processing units.
Task confirmed. Starting download..
25.0 %
50.0 %
75.0 %
100.0 % completed.


,re_tweeter,links,author,text,hashtags,reply_count,associated_tweet,timestamp,mentions,author_id,retweet_count,type,id,favorite_count
0,,[https:\/\/t.co\/WGlxA4Ak7Q],CoinJournal,#Markets #Etoro Where to buy Ripple as XRP gea...,,,,1634135588,,,,Tweet,1448295744524328960,
1,,[https:\/\/t.co\/TFusSmwP1W],CoinJournal,#Markets #eToro Where to buy Maker as MKR sees...,,,,1634134712,,,,Tweet,1448292073669414912,
2,,[https:\/\/t.co\/P2cnZkQ5Rq],CoinJournal,#Markets #eToro Where to buy TRON as TRX sees ...,,,,1634131653,,,,Tweet,1448279240630104064,
3,,[https:\/\/t.co\/efeX5JO8WR],CoinJournal,#Markets #eToro Where to buy Binance Coin as B...,,,,1634131113,,,,Tweet,1448276976213508096,
4,,[https:\/\/t.co\/1L04qPbKtA],CoinJournal,#Markets #Bitcoin US is now the top destinatio...,,,,1634129464,,,,Tweet,1448270060699586560,


### бленд 2 вариант

In [35]:
blend_source = {
                'id_dataset':'5ea20b4e95162936348f141d',
                'feature' : 'text'
            }

# Now, let's 'timeBlend' it to our dataset
df_blend_v2 = OpenBlender.timeBlend( token = token,
                                  anchor_ts = df.timestamp,
                                  blend_source = blend_source,
                                  blend_type = 'agg_in_intervals',
                                  interval_size = 60 * 60 * 24,
                                  direction = 'time_prior',
                                  interval_output = 'list',
                                  missing_values = 'raw')

Task ID: '6167ffaf0895fafb4a9d8d4c'.
Total estimated consumption: 1555.2 processing units.
Task confirmed. Starting download..
100%


In [36]:
df = pd.concat([df, df_blend_v2.loc[:, df_blend_v2.columns != 'timestamp']], axis = 1)
df.head()

,date,timestamp,price,target,BITCOIN_NE.text_COUNT_last1days,BITCOIN_NE.text_last1days,COINJOURNA.text_last1days,COINJOURNA.text_COUNT_last1days
0,01-01-2020 17:00:00,1.577898e+09,7212.7,1,3,[etoro ceo yoni assia on reaching 12 million u...,[],0
1,02-01-2020 17:00:00,1.577984e+09,6989.4,0,7,[government confirms crypto profits not taxabl...,[],0
2,03-01-2020 17:00:00,1.578071e+09,7367.5,1,8,[ticker tool uses the bch blockchain to provid...,[],0
3,04-01-2020 17:00:00,1.578157e+09,7369.8,0,6,[crypto fundraising is changing again in 2020 ...,[],0
4,05-01-2020 17:00:00,1.578244e+09,7364.6,0,5,[subhash chandra garg on the future of crypto ...,[],0


In [37]:
df_blend_v2['COINJOURNA.text_COUNT_last1days'].mean()

0.5550847457627118

In [38]:
blend_source = {
                'id_dataset':'5ea209c495162936348f13eb',
                'feature' : 'text'
            }

# Now, let's 'timeBlend' it to our dataset
df_blend_v3 = OpenBlender.timeBlend( token = token,
                                  anchor_ts = df.timestamp,
                                  blend_source = blend_source,
                                  blend_type = 'agg_in_intervals',
                                  interval_size = 60 * 60 * 24,
                                  direction = 'time_prior',
                                  interval_output = 'list',
                                  missing_values = 'raw')
df_blend_v3

Task ID: '6167ffbb0895fafb4a9d8d4c'.
Total estimated consumption: 1555.2 processing units.
Task confirmed. Starting download..
100%


,timestamp,CRYPTOCURR.text_COUNT_last1days,CRYPTOCURR.text_last1days
0,1577898000,1,[what are your 2020 predictions for #crypto?]
1,1577984400,0,[]
2,1578070800,2,[people are going to opt for something that is...
3,1578157200,1,[bullish on bitcoin]
4,1578243600,1,[a burger king location in venezuela is now ac...
...,...,...,...
231,1598202000,1,[you gotta have some fortitude to handle the s...
232,1598374800,0,[]
233,1598461200,0,[]
234,1598547600,0,[]


In [39]:
df_blend_v3['CRYPTOCURR.text_COUNT_last1days'].mean()

1.1610169491525424

In [40]:
df = pd.concat([df, df_blend_v3.loc[:, df_blend_v3.columns != 'timestamp']], axis = 1)
df.head()

,date,timestamp,price,target,BITCOIN_NE.text_COUNT_last1days,BITCOIN_NE.text_last1days,COINJOURNA.text_last1days,COINJOURNA.text_COUNT_last1days,CRYPTOCURR.text_COUNT_last1days,CRYPTOCURR.text_last1days
0,01-01-2020 17:00:00,1.577898e+09,7212.7,1,3,[etoro ceo yoni assia on reaching 12 million u...,[],0,1,[what are your 2020 predictions for #crypto?]
1,02-01-2020 17:00:00,1.577984e+09,6989.4,0,7,[government confirms crypto profits not taxabl...,[],0,0,[]
2,03-01-2020 17:00:00,1.578071e+09,7367.5,1,8,[ticker tool uses the bch blockchain to provid...,[],0,2,[people are going to opt for something that is...
3,04-01-2020 17:00:00,1.578157e+09,7369.8,0,6,[crypto fundraising is changing again in 2020 ...,[],0,1,[bullish on bitcoin]
4,05-01-2020 17:00:00,1.578244e+09,7364.6,0,5,[subhash chandra garg on the future of crypto ...,[],0,1,[a burger king location in venezuela is now ac...


In [41]:
blend_source = {
                'id_dataset':'5ea20cd09516293f790c119c',
                'feature' : 'text'
            }

# Now, let's 'timeBlend' it to our dataset
df_blend_v4 = OpenBlender.timeBlend( token = token,
                                  anchor_ts = df.timestamp,
                                  blend_source = blend_source,
                                  blend_type = 'agg_in_intervals',
                                  interval_size = 60 * 60 * 24,
                                  direction = 'time_prior',
                                  interval_output = 'list',
                                  missing_values = 'raw')
df_blend_v4

Task ID: '6167ffc30895fafb4a9d8d4c'.
Total estimated consumption: 1555.2 processing units.
Task confirmed. Starting download..
100%


,timestamp,CRYPTOCURR.text_COUNT_last1days,CRYPTOCURR.text_last1days
0,1577898000,2,"[line79, youtube13eamt4 indicator fx ]"
1,1577984400,5,"[23mt4 ea fx , line81, line82, 24mt4 ea fx ..."
2,1578070800,2,"[youtube14eamt4 indicator fx , line83]"
3,1578157200,7,"[mt4 fx , 19ea, line85, line87, youtube17mt4 ..."
4,1578243600,4,"[, line93, macdmt4 fx , line91]"
...,...,...,...
231,1598202000,0,[]
232,1598374800,0,[]
233,1598461200,1,[defi pieformance]
234,1598547600,0,[]


In [42]:
df_blend_v4['CRYPTOCURR.text_COUNT_last1days'].mean()

0.8813559322033898

In [43]:
df = pd.concat([df, df_blend_v4.loc[:, df_blend_v4.columns != 'timestamp']], axis = 1)
df.head()

,date,timestamp,price,target,BITCOIN_NE.text_COUNT_last1days,BITCOIN_NE.text_last1days,COINJOURNA.text_last1days,COINJOURNA.text_COUNT_last1days,CRYPTOCURR.text_COUNT_last1days,CRYPTOCURR.text_last1days,CRYPTOCURR.text_COUNT_last1days,CRYPTOCURR.text_last1days
0,01-01-2020 17:00:00,1.577898e+09,7212.7,1,3,[etoro ceo yoni assia on reaching 12 million u...,[],0,1,[what are your 2020 predictions for #crypto?],2,"[line79, youtube13eamt4 indicator fx ]"
1,02-01-2020 17:00:00,1.577984e+09,6989.4,0,7,[government confirms crypto profits not taxabl...,[],0,0,[],5,"[23mt4 ea fx , line81, line82, 24mt4 ea fx ..."
2,03-01-2020 17:00:00,1.578071e+09,7367.5,1,8,[ticker tool uses the bch blockchain to provid...,[],0,2,[people are going to opt for something that is...,2,"[youtube14eamt4 indicator fx , line83]"
3,04-01-2020 17:00:00,1.578157e+09,7369.8,0,6,[crypto fundraising is changing again in 2020 ...,[],0,1,[bullish on bitcoin],7,"[mt4 fx , 19ea, line85, line87, youtube17mt4 ..."
4,05-01-2020 17:00:00,1.578244e+09,7364.6,0,5,[subhash chandra garg on the future of crypto ...,[],0,1,[a burger king location in venezuela is now ac...,4,"[, line93, macdmt4 fx , line91]"


### тут немного другая штука - количество поисков

In [44]:
blend_source = {
                'id_dataset':'6114e0ed9516295907e7f5d4',
                'feature' : 'bitcoin_search'
            }

# Now, let's 'timeBlend' it to our dataset
df_blend_v5 = OpenBlender.timeBlend( token = token,
                                  anchor_ts = df.timestamp,
                                  blend_source = blend_source,
                                  blend_type = 'agg_in_intervals',
                                  interval_size = 60 * 60 * 24,
                                  direction = 'time_prior',
                                  interval_output = 'list',
                                  missing_values = 'raw')
df_blend_v5

Task ID: '6167ffd30895fafb4a9d8d4c'.
Total estimated consumption: 1555.2 processing units.
Task confirmed. Starting download..
100%


,BTC_FEARGR.bitcoin_search_COUNT_last1days,timestamp,BTC_FEARGR.bitcoin_search_last1days
0,1,1577898000,[14]
1,1,1577984400,[14]
2,1,1578070800,[14]
3,1,1578157200,[14]
4,1,1578243600,[14]
...,...,...,...
231,1,1598202000,[14]
232,1,1598374800,[15]
233,1,1598461200,[15]
234,1,1598547600,[15]


In [45]:
df_blend_v5 = df_blend_v5.drop(['BTC_FEARGR.bitcoin_search_COUNT_last1days'], axis=1)

In [46]:
df_blend_v5['BTC_FEARGR.bitcoin_search_last1days'] = df_blend_v5['BTC_FEARGR.bitcoin_search_last1days'].apply(lambda x: x[0])

In [47]:
df = pd.concat([df, df_blend_v5.loc[:, df_blend_v5.columns != 'timestamp']], axis = 1)
df.head()

,date,timestamp,price,target,BITCOIN_NE.text_COUNT_last1days,BITCOIN_NE.text_last1days,COINJOURNA.text_last1days,COINJOURNA.text_COUNT_last1days,CRYPTOCURR.text_COUNT_last1days,CRYPTOCURR.text_last1days,CRYPTOCURR.text_COUNT_last1days,CRYPTOCURR.text_last1days,BTC_FEARGR.bitcoin_search_last1days
0,01-01-2020 17:00:00,1.577898e+09,7212.7,1,3,[etoro ceo yoni assia on reaching 12 million u...,[],0,1,[what are your 2020 predictions for #crypto?],2,"[line79, youtube13eamt4 indicator fx ]",14
1,02-01-2020 17:00:00,1.577984e+09,6989.4,0,7,[government confirms crypto profits not taxabl...,[],0,0,[],5,"[23mt4 ea fx , line81, line82, 24mt4 ea fx ...",14
2,03-01-2020 17:00:00,1.578071e+09,7367.5,1,8,[ticker tool uses the bch blockchain to provid...,[],0,2,[people are going to opt for something that is...,2,"[youtube14eamt4 indicator fx , line83]",14
3,04-01-2020 17:00:00,1.578157e+09,7369.8,0,6,[crypto fundraising is changing again in 2020 ...,[],0,1,[bullish on bitcoin],7,"[mt4 fx , 19ea, line85, line87, youtube17mt4 ...",14
4,05-01-2020 17:00:00,1.578244e+09,7364.6,0,5,[subhash chandra garg on the future of crypto ...,[],0,1,[a burger king location in venezuela is now ac...,4,"[, line93, macdmt4 fx , line91]",14


In [48]:
%%time

blend_source = {
                'id_dataset':'5defce899516296bfe37c366',
                'feature' : 'title'
            }

# Now, let's 'timeBlend' it to our dataset
df_blend_v6 = OpenBlender.timeBlend( token = token,
                                  anchor_ts = df.timestamp,
                                  blend_source = blend_source,
                                  blend_type = 'agg_in_intervals',
                                  interval_size = 60 * 60 * 24,
                                  direction = 'time_prior',
                                  interval_output = 'list',
                                  missing_values = 'raw')
df_blend_v6

Task ID: '6167ffe80895fafb4a9d8d4c'.
Total estimated consumption: 1555.2 processing units.
Task confirmed. Starting download..
100%
CPU times: user 2.74 s, sys: 13.9 ms, total: 2.76 s
Wall time: 11 s


,timestamp,FOX_NEWS_B.title_COUNT_last1days,FOX_NEWS_B.title_last1days
0,1577898000,84,[former nissan chairman flees the country whil...
1,1577984400,76,[why january is the most popular time to divor...
2,1578070800,160,[ncaa athlete pay debate: why a political show...
3,1578157200,92,[state posts staggering sales figures after fi...
4,1578243600,35,[this 2020 billionaire just came one step clos...
...,...,...,...
231,1598202000,0,[]
232,1598374800,0,[]
233,1598461200,16,[brother of suzanne morphew pleads for brother...
234,1598547600,0,[]


In [51]:
%%time

blend_source = {
                'id_dataset':'5defce899516296bfe37c366',
                'feature' : 'headline'
            }

# Now, let's 'timeBlend' it to our dataset
df_blend_v7 = OpenBlender.timeBlend( token = token,
                                  anchor_ts = df.timestamp,
                                  blend_source = blend_source,
                                  blend_type = 'agg_in_intervals',
                                  interval_size = 60 * 60 * 24,
                                  direction = 'time_prior',
                                  interval_output = 'list',
                                  missing_values = 'raw')
df_blend_v7

Task ID: '616800180895fafb4a9d8d4c'.
Total estimated consumption: 1555.2 processing units.
Task confirmed. Starting download..
100%
CPU times: user 2.01 s, sys: 10.6 ms, total: 2.02 s
Wall time: 7.27 s


,FOX_NEWS_B.headline_COUNT_last1days,timestamp,FOX_NEWS_B.headline_last1days
0,59,1577898000,[ryan patel of the drucker school of managemen...
1,60,1577984400,[will 2020 bring even more stock market gains?...
2,92,1578070800,"[, cars her way founder lisa copeland discusse..."
3,49,1578157200,"[, the airstrikes were targeted two vehicles c..."
4,27,1578243600,"[, foxnews, 2020 democratic candidate tom stey..."
...,...,...,...
231,0,1598202000,[]
232,0,1598374800,[]
233,13,1598461200,"[, rnc chairwoman ronna mcdaniel on what to ex..."
234,0,1598547600,[]


In [52]:
df = pd.concat([df, df_blend_v6.loc[:, df_blend_v6.columns != 'timestamp']], axis = 1)
df.head()

,date,timestamp,price,target,BITCOIN_NE.text_COUNT_last1days,BITCOIN_NE.text_last1days,COINJOURNA.text_last1days,COINJOURNA.text_COUNT_last1days,CRYPTOCURR.text_COUNT_last1days,CRYPTOCURR.text_last1days,CRYPTOCURR.text_COUNT_last1days,CRYPTOCURR.text_last1days,BTC_FEARGR.bitcoin_search_last1days,FOX_NEWS_B.title_COUNT_last1days,FOX_NEWS_B.title_last1days
0,01-01-2020 17:00:00,1.577898e+09,7212.7,1,3,[etoro ceo yoni assia on reaching 12 million u...,[],0,1,[what are your 2020 predictions for #crypto?],2,"[line79, youtube13eamt4 indicator fx ]",14,84,[former nissan chairman flees the country whil...
1,02-01-2020 17:00:00,1.577984e+09,6989.4,0,7,[government confirms crypto profits not taxabl...,[],0,0,[],5,"[23mt4 ea fx , line81, line82, 24mt4 ea fx ...",14,76,[why january is the most popular time to divor...
2,03-01-2020 17:00:00,1.578071e+09,7367.5,1,8,[ticker tool uses the bch blockchain to provid...,[],0,2,[people are going to opt for something that is...,2,"[youtube14eamt4 indicator fx , line83]",14,160,[ncaa athlete pay debate: why a political show...
3,04-01-2020 17:00:00,1.578157e+09,7369.8,0,6,[crypto fundraising is changing again in 2020 ...,[],0,1,[bullish on bitcoin],7,"[mt4 fx , 19ea, line85, line87, youtube17mt4 ...",14,92,[state posts staggering sales figures after fi...
4,05-01-2020 17:00:00,1.578244e+09,7364.6,0,5,[subhash chandra garg on the future of crypto ...,[],0,1,[a burger king location in venezuela is now ac...,4,"[, line93, macdmt4 fx , line91]",14,35,[this 2020 billionaire just came one step clos...


In [53]:
df = pd.concat([df, df_blend_v7.loc[:, df_blend_v7.columns != 'timestamp']], axis = 1)
df.head()

,date,timestamp,price,target,BITCOIN_NE.text_COUNT_last1days,BITCOIN_NE.text_last1days,COINJOURNA.text_last1days,COINJOURNA.text_COUNT_last1days,CRYPTOCURR.text_COUNT_last1days,CRYPTOCURR.text_last1days,CRYPTOCURR.text_COUNT_last1days,CRYPTOCURR.text_last1days,BTC_FEARGR.bitcoin_search_last1days,FOX_NEWS_B.title_COUNT_last1days,FOX_NEWS_B.title_last1days,FOX_NEWS_B.headline_COUNT_last1days,FOX_NEWS_B.headline_last1days
0,01-01-2020 17:00:00,1.577898e+09,7212.7,1,3,[etoro ceo yoni assia on reaching 12 million u...,[],0,1,[what are your 2020 predictions for #crypto?],2,"[line79, youtube13eamt4 indicator fx ]",14,84,[former nissan chairman flees the country whil...,59,[ryan patel of the drucker school of managemen...
1,02-01-2020 17:00:00,1.577984e+09,6989.4,0,7,[government confirms crypto profits not taxabl...,[],0,0,[],5,"[23mt4 ea fx , line81, line82, 24mt4 ea fx ...",14,76,[why january is the most popular time to divor...,60,[will 2020 bring even more stock market gains?...
2,03-01-2020 17:00:00,1.578071e+09,7367.5,1,8,[ticker tool uses the bch blockchain to provid...,[],0,2,[people are going to opt for something that is...,2,"[youtube14eamt4 indicator fx , line83]",14,160,[ncaa athlete pay debate: why a political show...,92,"[, cars her way founder lisa copeland discusse..."
3,04-01-2020 17:00:00,1.578157e+09,7369.8,0,6,[crypto fundraising is changing again in 2020 ...,[],0,1,[bullish on bitcoin],7,"[mt4 fx , 19ea, line85, line87, youtube17mt4 ...",14,92,[state posts staggering sales figures after fi...,49,"[, the airstrikes were targeted two vehicles c..."
4,05-01-2020 17:00:00,1.578244e+09,7364.6,0,5,[subhash chandra garg on the future of crypto ...,[],0,1,[a burger king location in venezuela is now ac...,4,"[, line93, macdmt4 fx , line91]",14,35,[this 2020 billionaire just came one step clos...,27,"[, foxnews, 2020 democratic candidate tom stey..."


### дамп так как тексты неверно записываются в файл

In [54]:
import pickle
with open("full_df_with_7_sources.pickle", "wb") as f:
    pickle.dump(df, f)

In [65]:
with open("full_df_with_7_sources.pickle", "rb") as f:
    df = pickle.load(f)
d

,date,timestamp,price,target,BITCOIN_NE.text_COUNT_last1days,BITCOIN_NE.text_last1days,COINJOURNA.text_last1days,COINJOURNA.text_COUNT_last1days,CRYPTOCURR.text_COUNT_last1days,CRYPTOCURR.text_last1days,CRYPTOCURR.text_COUNT_last1days,CRYPTOCURR.text_last1days,BTC_FEARGR.bitcoin_search_last1days,FOX_NEWS_B.title_COUNT_last1days,FOX_NEWS_B.title_last1days,FOX_NEWS_B.headline_COUNT_last1days,FOX_NEWS_B.headline_last1days
0,01-01-2020 17:00:00,1.577898e+09,7212.7,1,3,[etoro ceo yoni assia on reaching 12 million u...,[],0,1,[what are your 2020 predictions for #crypto?],2,"[line79, youtube13eamt4 indicator fx ]",14,84,[former nissan chairman flees the country whil...,59,[ryan patel of the drucker school of managemen...
1,02-01-2020 17:00:00,1.577984e+09,6989.4,0,7,[government confirms crypto profits not taxabl...,[],0,0,[],5,"[23mt4 ea fx , line81, line82, 24mt4 ea fx ...",14,76,[why january is the most popular time to divor...,60,[will 2020 bring even more stock market gains?...
2,03-01-2020 17:00:00,1.578071e+09,7367.5,1,8,[ticker tool uses the bch blockchain to provid...,[],0,2,[people are going to opt for something that is...,2,"[youtube14eamt4 indicator fx , line83]",14,160,[ncaa athlete pay debate: why a political show...,92,"[, cars her way founder lisa copeland discusse..."
3,04-01-2020 17:00:00,1.578157e+09,7369.8,0,6,[crypto fundraising is changing again in 2020 ...,[],0,1,[bullish on bitcoin],7,"[mt4 fx , 19ea, line85, line87, youtube17mt4 ...",14,92,[state posts staggering sales figures after fi...,49,"[, the airstrikes were targeted two vehicles c..."
4,05-01-2020 17:00:00,1.578244e+09,7364.6,0,5,[subhash chandra garg on the future of crypto ...,[],0,1,[a burger king location in venezuela is now ac...,4,"[, line93, macdmt4 fx , line91]",14,35,[this 2020 billionaire just came one step clos...,27,"[, foxnews, 2020 democratic candidate tom stey..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,23-08-2020 17:00:00,1.598202e+09,11663.0,0,0,[],[],0,1,[you gotta have some fortitude to handle the s...,0,[],14,0,[],0,[]
232,25-08-2020 17:00:00,1.598375e+09,11341.0,0,0,[],[xrps active wallets hit 7day high despite 92 ...,1,0,[],0,[],15,0,[],0,[]
233,26-08-2020 17:00:00,1.598461e+09,11471.0,1,0,[],[the irs is sending tax warning letters to cry...,3,0,[],1,[defi pieformance],15,16,[brother of suzanne morphew pleads for brother...,13,"[, rnc chairwoman ronna mcdaniel on what to ex..."
234,27-08-2020 17:00:00,1.598548e+09,11343.0,0,0,[],[brazils central bank considering the developm...,4,0,[],0,[],15,0,[],0,[]


In [57]:
d['BITCOIN_NE.text_last1days'][0]

['etoro ceo yoni assia on reaching 12 million users and why cryptos are a gateway to stocks ',
 'china takes another step away from usd hegemony ',
 'veriblock captured close to 60 of btcs op return transactions in 2019 ']

In [59]:
pd.read_csv("full_df_with_7_sources.csv")['BITCOIN_NE.text_last1days'][0]

"['etoro ceo yoni assia on reaching 12 million users and why cryptos are a gateway to stocks ', 'china takes another step away from usd hegemony ', 'veriblock captured close to 60 of btcs op return transactions in 2019 ']"

### csv 

In [60]:
# df.to_csv("full_df_with_7_sources.csv", index=False)

In [66]:
df.columns

Index(['date', 'timestamp', 'price', 'target',
       'BITCOIN_NE.text_COUNT_last1days', 'BITCOIN_NE.text_last1days',
       'COINJOURNA.text_last1days', 'COINJOURNA.text_COUNT_last1days',
       'CRYPTOCURR.text_COUNT_last1days', 'CRYPTOCURR.text_last1days',
       'CRYPTOCURR.text_COUNT_last1days', 'CRYPTOCURR.text_last1days',
       'BTC_FEARGR.bitcoin_search_last1days',
       'FOX_NEWS_B.title_COUNT_last1days', 'FOX_NEWS_B.title_last1days',
       'FOX_NEWS_B.headline_COUNT_last1days', 'FOX_NEWS_B.headline_last1days'],
      dtype='object')

In [67]:
df['CRYPTOCURR.text_last1days_v1'] = df['CRYPTOCURR.text_last1days'].values[:, 0]
df['CRYPTOCURR.text_last1days_v2'] = df['CRYPTOCURR.text_last1days'].values[:, 1]

In [68]:
df['CRYPTOCURR.text_COUNT_last1days_v1'] = df['CRYPTOCURR.text_COUNT_last1days'].values[:, 0]
df['CRYPTOCURR.text_COUNT_last1days_v2'] = df['CRYPTOCURR.text_COUNT_last1days'].values[:, 1]

del df['CRYPTOCURR.text_last1days']
del df['CRYPTOCURR.text_COUNT_last1days']

In [71]:
count_features = [
    'BITCOIN_NE.text_COUNT_last1days', 
    'COINJOURNA.text_COUNT_last1days', 
    'CRYPTOCURR.text_COUNT_last1days_v1', 
    'CRYPTOCURR.text_COUNT_last1days_v2',
    'BTC_FEARGR.bitcoin_search_last1days', 
    'FOX_NEWS_B.title_COUNT_last1days', 
    'FOX_NEWS_B.headline_COUNT_last1days'
]
text_features = [
    'BITCOIN_NE.text_last1days', 'CRYPTOCURR.text_last1days_v1',
    'CRYPTOCURR.text_last1days_v2', 'FOX_NEWS_B.title_last1days', 
    'FOX_NEWS_B.headline_last1days', 'COINJOURNA.text_last1days'
]

In [72]:
df['BTC_FEARGR.bitcoin_search_last1days'] = df['BTC_FEARGR.bitcoin_search_last1days'].apply(lambda x: int(x))

### видим что у количества корреляция очень слабая

In [73]:
df[['target']+count_features].corr()['target']

target                                 1.000000
BITCOIN_NE.text_COUNT_last1days        0.015667
COINJOURNA.text_COUNT_last1days        0.031513
CRYPTOCURR.text_COUNT_last1days_v1    -0.106928
CRYPTOCURR.text_COUNT_last1days_v2    -0.048718
BTC_FEARGR.bitcoin_search_last1days    0.037524
FOX_NEWS_B.title_COUNT_last1days      -0.080625
FOX_NEWS_B.headline_COUNT_last1days   -0.069817
Name: target, dtype: float64

In [74]:
df['count_sum'] = df[count_features].sum(axis=1)

In [75]:
df[['count_sum', 'target']].corr()['target']

count_sum   -0.076851
target       1.000000
Name: target, dtype: float64

In [77]:
row_texts = list()
for row in df[text_features].iterrows():
    row_text = list()
    for val in row[1].values:
        for v in val:
            row_text.append(v)
    row_texts.append(row_text)

In [78]:
df['text_full'] = row_texts

In [82]:
# df.to_csv("full_df_with_7_sources_and_summary.csv", index=False)

with open("full_df_with_7_sources_and_summary.pickle", "wb") as f:
    pickle.dump(df, f)

### finbert api

In [83]:
import requests
API_TOKEN = "your_token"
API_URL = "https://api-inference.huggingface.co/models/ProsusAI/finbert"
headers = {"Authorization": f"Bearer {API_TOKEN}"}

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

output = query({"inputs": "I like you. I love you"})
output

[[{'label': 'positive', 'score': 0.08782877773046494},
  {'label': 'negative', 'score': 0.021379711106419563},
  {'label': 'neutral', 'score': 0.8907914757728577}]]

In [135]:
%%time
from tqdm import tqdm
results=list()
for v in tqdm(df['text_full'].values):
    r = query(v)
    results.append(r)

100% 236/236 [15:06<00:00,  3.84s/it] 

CPU times: user 10.1 s, sys: 810 ms, total: 10.9 s
Wall time: 15min 6s


### видим что здесь слабая корреляция, если считать скор сразу по всем

In [141]:
features = ['target', 'res_pos', 'res_neg']
df[features].dropna().corr()['target']

target     1.000000
res_pos    0.036260
res_neg    0.011118
Name: target, dtype: float64

In [ ]:
# %%time

# df['res_pos'] = pd.Series()
# df['res_neg'] = pd.Series()
# df['res_neutral'] = pd.Series()
# for i in tqdm(range(236)):
#     print(i)
#     sub_df = df.loc[i, :]#['BITCOIN_NE.text_last1days']
#     text_length = len(sub_df['text_full'])
#     res_pos = 0
#     res_neg = 0
#     res_neu = 0
#     for j in range(text_length):
#         try:
#             current_res = query(sub_df['text_full'][j])[0]
#             current_res_pos = current_res[0]['score']
#             current_res_neg = current_res[1]['score']
#             current_res_neutral = current_res[2]['score']
#             if current_res_pos>=current_res_neg:
#                 if current_res_pos > current_res_neutral:
#                     res_pos += 1
#                 else:
#                     res_neu += 1
#             else:
#                 if current_res_neg > current_res_neutral:
#                     res_neg += 1
#                 else:
#                     res_neu += 1
#         except Exception as e:
#             continue
#     df.loc[i, 'res_pos'] = res_pos
#     df.loc[i, 'res_neg'] = res_neg
#     df.loc[i, 'res_neutral'] = res_neu
# df

### тут и позитив и негатив и нейтрал

In [146]:
features = ['target', 'res_pos', 'res_neg']
df[features].dropna().corr()['target']

target     1.000000
res_pos   -0.009399
res_neg    0.000386
Name: target, dtype: float64

In [148]:
features = ['target', 'res_pos', 'res_neg', 'res_neu']
df[features].dropna().corr()['target']

target     1.000000
res_pos    0.113342
res_neg   -0.078145
res_neu   -0.032336
Name: target, dtype: float64

### только по биткоин новостям и добавляю вероятности по дню с каждой новости

In [84]:
%%time
from tqdm import tqdm

df['res_pos'] = pd.Series()
df['res_neg'] = pd.Series()
df['res_neu'] = pd.Series()
for i in tqdm(range(236)):
    print(i)
    sub_df = df.loc[i, :]#['BITCOIN_NE.text_last1days']
    text_length = len(sub_df['BITCOIN_NE.text_last1days'])
    res_pos = 0
    res_neg = 0
    res_neu = 0
    for j in range(text_length):
        try:
            q = query(sub_df['BITCOIN_NE.text_last1days'][j])
            current_res = q[0]
            current_res_pos = current_res[0]['score']
            current_res_neg = current_res[1]['score']
            current_res_neutral = current_res[2]['score']
            res_pos += current_res_pos
            res_neg += current_res_neg
            res_neu += current_res_neutral
            # for counting
#             if current_res_pos>=current_res_neg:
#                 if current_res_pos > current_res_neutral:
#                     res_pos += 1
#                 else:
#                     res_neu += 1
#             else:
#                 if current_res_neg > current_res_neutral:
#                     res_neg += 1
#                 else:
#                     res_neu += 1
        except Exception as e:
            continue
    df.loc[i, 'res_pos'] = res_pos
    df.loc[i, 'res_neg'] = res_neg
    df.loc[i, 'res_neu'] = res_neu
df

<timed exec>:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
<timed exec>:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
<timed exec>:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  0% 0/236 [00:00<?, ?it/s]

0


  0% 1/236 [00:02<09:08,  2.33s/it]

1


  1% 2/236 [00:07<15:20,  3.93s/it]

...

 95% 224/236 [21:36<01:08,  5.68s/it]

224


100% 236/236 [21:36<00:00,  5.50s/it]

225
226
227
228
229
230
231
232
233
234
235
CPU times: user 42.8 s, sys: 2.34 s, total: 45.1 s
Wall time: 21min 36s


,date,timestamp,price,target,BITCOIN_NE.text_COUNT_last1days,BITCOIN_NE.text_last1days,COINJOURNA.text_last1days,COINJOURNA.text_COUNT_last1days,BTC_FEARGR.bitcoin_search_last1days,FOX_NEWS_B.title_COUNT_last1days,...,FOX_NEWS_B.headline_last1days,CRYPTOCURR.text_last1days_v1,CRYPTOCURR.text_last1days_v2,CRYPTOCURR.text_COUNT_last1days_v1,CRYPTOCURR.text_COUNT_last1days_v2,count_sum,text_full,res_pos,res_neg,res_neu
0,01-01-2020 17:00:00,1.577898e+09,7212.7,1,3,[etoro ceo yoni assia on reaching 12 million u...,[],0,14,84,...,[ryan patel of the drucker school of managemen...,[what are your 2020 predictions for #crypto?],"[line79, youtube13eamt4 indicator fx ]",1,2,163,[etoro ceo yoni assia on reaching 12 million u...,0.343571,0.631427,2.025002
1,02-01-2020 17:00:00,1.577984e+09,6989.4,0,7,[government confirms crypto profits not taxabl...,[],0,14,76,...,[will 2020 bring even more stock market gains?...,[],"[23mt4 ea fx , line81, line82, 24mt4 ea fx ...",0,5,162,[government confirms crypto profits not taxabl...,1.525628,2.151473,3.322899
2,03-01-2020 17:00:00,1.578071e+09,7367.5,1,8,[ticker tool uses the bch blockchain to provid...,[],0,14,160,...,"[, cars her way founder lisa copeland discusse...",[people are going to opt for something that is...,"[youtube14eamt4 indicator fx , line83]",2,2,278,[ticker tool uses the bch blockchain to provid...,0.790071,0.639061,6.570868
3,04-01-2020 17:00:00,1.578157e+09,7369.8,0,6,[crypto fundraising is changing again in 2020 ...,[],0,14,92,...,"[, the airstrikes were targeted two vehicles c...",[bullish on bitcoin],"[mt4 fx , 19ea, line85, line87, youtube17mt4 ...",1,7,169,[crypto fundraising is changing again in 2020 ...,1.064533,1.109780,3.825687
4,05-01-2020 17:00:00,1.578244e+09,7364.6,0,5,[subhash chandra garg on the future of crypto ...,[],0,14,35,...,"[, foxnews, 2020 democratic candidate tom stey...",[a burger king location in venezuela is now ac...,"[, line93, macdmt4 fx , line91]",1,4,86,[subhash chandra garg on the future of crypto ...,1.266352,0.183858,3.549790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,23-08-2020 17:00:00,1.598202e+09,11663.0,0,0,[],[],0,14,0,...,[],[you gotta have some fortitude to handle the s...,[],1,0,15,[you gotta have some fortitude to handle the s...,0.000000,0.000000,0.000000
232,25-08-2020 17:00:00,1.598375e+09,11341.0,0,0,[],[xrps active wallets hit 7day high despite 92 ...,1,15,0,...,[],[],[],0,0,16,[xrps active wallets hit 7day high despite 92 ...,0.000000,0.000000,0.000000
233,26-08-2020 17:00:00,1.598461e+09,11471.0,1,0,[],[the irs is sending tax warning letters to cry...,3,15,16,...,"[, rnc chairwoman ronna mcdaniel on what to ex...",[],[defi pieformance],0,1,48,"[defi pieformance, brother of suzanne morphew ...",0.000000,0.000000,0.000000
234,27-08-2020 17:00:00,1.598548e+09,11343.0,0,0,[],[brazils central bank considering the developm...,4,15,0,...,[],[],[],0,0,19,[brazils central bank considering the developm...,0.000000,0.000000,0.000000


In [86]:
features = ['target', 'res_pos', 'res_neg', 'res_neu']
df[features].dropna().corr()['target']

target     1.000000
res_pos    0.048596
res_neg   -0.001936
res_neu   -0.052912
Name: target, dtype: float64

### только позитив и негатив с количеством

In [87]:
%%time
from tqdm import tqdm

df['res_pos_count'] = pd.Series()
df['res_neg_count'] = pd.Series()
# df['res_neu'] = pd.Series()
for i in tqdm(range(236)):
    print(i)
    sub_df = df.loc[i, :]#['BITCOIN_NE.text_last1days']
    text_length = len(sub_df['BITCOIN_NE.text_last1days'])
    res_pos = 0
    res_neg = 0
#     res_neu = 0
    for j in range(text_length):
        try:
            q = query(sub_df['BITCOIN_NE.text_last1days'][j])
            current_res = q[0]
            current_res_pos = current_res[0]['score']
            current_res_neg = current_res[1]['score']
#             current_res_neutral = current_res[2]['score']
            # for counting
            if current_res_pos>=current_res_neg:
#                 if current_res_pos > current_res_neutral:
                res_pos += 1
#                 else:
#                 res_neu += 1
            else:
#                 if current_res_neg > current_res_neutral:
                res_neg += 1
#                 else:
#                 res_neu += 1
        except Exception as e:
            continue
    df.loc[i, 'res_pos_count'] = res_pos
    df.loc[i, 'res_neg_count'] = res_neg
#     df.loc[i, 'res_neu'] = res_neu
df

<timed exec>:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
<timed exec>:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  0% 0/236 [00:00<?, ?it/s]

0


  0% 1/236 [00:02<08:50,  2.26s/it]

1


  1% 2/236 [00:07<15:27,  3.96s/it]

...

 95% 224/236 [16:19<01:06,  5.52s/it]

224


100% 236/236 [16:20<00:00,  4.16s/it]

225
226
227
228
229
230
231
232
233
234
235
CPU times: user 45.5 s, sys: 2.56 s, total: 48 s
Wall time: 16min 20s


,date,timestamp,price,target,BITCOIN_NE.text_COUNT_last1days,BITCOIN_NE.text_last1days,COINJOURNA.text_last1days,COINJOURNA.text_COUNT_last1days,BTC_FEARGR.bitcoin_search_last1days,FOX_NEWS_B.title_COUNT_last1days,...,CRYPTOCURR.text_last1days_v2,CRYPTOCURR.text_COUNT_last1days_v1,CRYPTOCURR.text_COUNT_last1days_v2,count_sum,text_full,res_pos,res_neg,res_neu,res_pos_count,res_neg_count
0,01-01-2020 17:00:00,1.577898e+09,7212.7,1,3,[etoro ceo yoni assia on reaching 12 million u...,[],0,14,84,...,"[line79, youtube13eamt4 indicator fx ]",1,2,163,[etoro ceo yoni assia on reaching 12 million u...,0.343571,0.631427,2.025002,2.0,1.0
1,02-01-2020 17:00:00,1.577984e+09,6989.4,0,7,[government confirms crypto profits not taxabl...,[],0,14,76,...,"[23mt4 ea fx , line81, line82, 24mt4 ea fx ...",0,5,162,[government confirms crypto profits not taxabl...,1.525628,2.151473,3.322899,2.0,5.0
2,03-01-2020 17:00:00,1.578071e+09,7367.5,1,8,[ticker tool uses the bch blockchain to provid...,[],0,14,160,...,"[youtube14eamt4 indicator fx , line83]",2,2,278,[ticker tool uses the bch blockchain to provid...,0.790071,0.639061,6.570868,5.0,3.0
3,04-01-2020 17:00:00,1.578157e+09,7369.8,0,6,[crypto fundraising is changing again in 2020 ...,[],0,14,92,...,"[mt4 fx , 19ea, line85, line87, youtube17mt4 ...",1,7,169,[crypto fundraising is changing again in 2020 ...,1.064533,1.109780,3.825687,5.0,1.0
4,05-01-2020 17:00:00,1.578244e+09,7364.6,0,5,[subhash chandra garg on the future of crypto ...,[],0,14,35,...,"[, line93, macdmt4 fx , line91]",1,4,86,[subhash chandra garg on the future of crypto ...,1.266352,0.183858,3.549790,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,23-08-2020 17:00:00,1.598202e+09,11663.0,0,0,[],[],0,14,0,...,[],1,0,15,[you gotta have some fortitude to handle the s...,0.000000,0.000000,0.000000,0.0,0.0
232,25-08-2020 17:00:00,1.598375e+09,11341.0,0,0,[],[xrps active wallets hit 7day high despite 92 ...,1,15,0,...,[],0,0,16,[xrps active wallets hit 7day high despite 92 ...,0.000000,0.000000,0.000000,0.0,0.0
233,26-08-2020 17:00:00,1.598461e+09,11471.0,1,0,[],[the irs is sending tax warning letters to cry...,3,15,16,...,[defi pieformance],0,1,48,"[defi pieformance, brother of suzanne morphew ...",0.000000,0.000000,0.000000,0.0,0.0
234,27-08-2020 17:00:00,1.598548e+09,11343.0,0,0,[],[brazils central bank considering the developm...,4,15,0,...,[],0,0,19,[brazils central bank considering the developm...,0.000000,0.000000,0.000000,0.0,0.0


In [88]:
features = ['target', 'res_pos_count', 'res_neg_count']
df[features].dropna().corr()['target']

target           1.000000
res_pos_count   -0.019491
res_neg_count   -0.014140
Name: target, dtype: float64

In [96]:
df[features][(df['res_pos_count']!=0)&(df['res_neg_count']!=0)].corr()['target']

target           1.000000
res_pos_count    0.005539
res_neg_count    0.015031
Name: target, dtype: float64

### full(уберу нейтральные ввиду того, что они не имеют смысла, как правило новость всё равно будет нейтральная)

In [97]:
%%time
from tqdm import tqdm

df['res_pos_full_count'] = pd.Series()
df['res_neg_full_count'] = pd.Series()
df['res_pos_full_prob'] = pd.Series()
df['res_neg_full_prob'] = pd.Series()
# df['res_neu_full_count'] = pd.Series()
for i in tqdm(range(236)):
    print(i)
    sub_df = df.loc[i, :]#['BITCOIN_NE.text_last1days']
    text_length = len(sub_df['text_full'])
    res_pos = 0
    res_pos_p = 0
    res_neg = 0
    res_neg_p = 0
    for j in range(text_length):
        try:
            q = query(sub_df['text_full'][j])
            current_res = q[0]
            current_res_pos = current_res[0]['score']
            current_res_neg = current_res[1]['score']
            if current_res_pos>=current_res_neg:
                res_pos += 1
                res_pos_p += current_res_pos
            else:
                res_neg += 1
                res_neg_p += current_res_neg
        except Exception as e:
            continue
    df.loc[i, 'res_pos_full_count'] = res_pos
    df.loc[i, 'res_neg_full_count'] = res_neg
    df.loc[i, 'res_pos_full_prob'] = res_pos_p
    df.loc[i, 'res_neg_full_prob'] = res_neg_p
df

<timed exec>:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
<timed exec>:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
<timed exec>:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
<timed exec>:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  0% 0/236 [00:00<?, ?it/s]

0


  0% 1/236 [01:54<7:29:35, 114.79s/it]

1


  1% 2/236 [03:40<7:05:53, 109.20s/it]

...

  4% 10/236 [26:18<12:24:52, 197.75s/it]

10


  5% 11/236 [29:07<9:55:53, 158.90s/it] 

11


KeyboardInterrupt: 

In [98]:
features = ['target', 'res_pos_full_count', 'res_neg_full_count', "res_pos_full_prob", "res_neg_full_prob"]
df[features].dropna().corr()['target']

target                1.000000
res_pos_full_count    0.231061
res_neg_full_count    0.064298
res_pos_full_prob     0.277992
res_neg_full_prob     0.055759
Name: target, dtype: float64